<a href="https://colab.research.google.com/github/daisyKim12/Tensorflow_Study/blob/main/Lecture_C3_New_Beans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# downgrade google colab tensorflow version to 2.9.0
import urllib.request

url = 'https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/tfcert.py'
urllib.request.urlretrieve(url, 'tfcert.py')
%run tfcert.py

import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
IMG_SIZE = 300

In [ ]:
def preprocess(image, label):
  #resize image into (300,300,3)
  #Normalize by rescaling by 255
  image = tf.image.resize(image, size=(300,300))
  image /= 255
  return image, label

In [ ]:
(ds_train, ds_validation), ds_info = tfds.load(
    name = 'beans',
    split=['train', 'validation'],
    as_supervised=True,
    with_info=True,
)

In [ ]:
BATCH_SIZE = 32

In [ ]:
ds_train = ds_train.map(preprocess).cache(
    ).shuffle(ds_info.splits['train'].num_examples).batch(BATCH_SIZE).prefetch(
    tf.data.experimental.AUTOTUNE)
ds_validation = ds_validation.map(preprocess).batch(BATCH_SIZE).cache(
    ).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
from tensorflow.keras.applications import VGG16

In [ ]:
transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(300,300,3))
transfer_model.trainable = False

58889256/58889256 [==============================] - 4s 0us/step


In [ ]:
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
model = tf.keras.models.Sequential([
    transfer_model,
    Flatten(),
    Dropout(0.40),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
checkpoint_path = 'tmp_checkpoint.ckpt'
checkpoint = ModelCheckpoint(filepath = checkpoint_path,
                             save_weights_only = True,
                             save_best_only = True,
                             monitor = 'val_loss',
                             verbose=1)

In [ ]:
model.fit(ds_train, validation_data=ds_validation, epochs=5, callbacks=[checkpoint])

Epoch 1/5
33/33 [==============================] - ETA: 0s - loss: 0.8896 - acc: 0.6470
Epoch 1: val_loss improved from inf to 0.44137, saving model to tmp_checkpoint.ckpt
33/33 [==============================] - 12s 308ms/step - loss: 0.8896 - acc: 0.6470 - val_loss: 0.4414 - val_acc: 0.7970
Epoch 2/5
33/33 [==============================] - ETA: 0s - loss: 0.2759 - acc: 0.8975
Epoch 2: val_loss did not improve from 0.44137
33/33 [==============================] - 9s 282ms/step - loss: 0.2759 - acc: 0.8975 - val_loss: 0.5960 - val_acc: 0.7368
Epoch 3/5
33/33 [==============================] - ETA: 0s - loss: 0.2590 - acc: 0.8888
Epoch 3: val_loss did not improve from 0.44137
33/33 [==============================] - 9s 283ms/step - loss: 0.2590 - acc: 0.8888 - val_loss: 0.6017 - val_acc: 0.7820
Epoch 4/5
33/33 [==============================] - ETA: 0s - loss: 0.1291 - acc: 0.9594
Epoch 4: val_loss improved from 0.44137 to 0.38825, saving model to tmp_checkpoint.ckpt
33/33 [===========

In [ ]:
model.load_weights(checkpoint_path)

In [ ]:
model.evaluate(ds_validation)

5/5 [==============================] - 1s 183ms/step - loss: 0.3741 - acc: 0.8571


[0.3740919828414917, 0.8571428656578064]